In [1]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [2]:
positive_reviews = [
    "I love this product! It works perfectly.",
    "Amazing quality and super fast delivery!",
    "Highly recommended, I will buy again.",
    "Five stars, exceeded my expectations.",
    "Great value for the price.",
    "Very satisfied with this purchase.",
    "Easy to use and very effective.",
    "Excellent performance and build quality.",
    "Superb customer service and fast shipping.",
    "This is the best thing I've bought all year!"
]

negative_reviews = [
    "Terrible product. Do not buy.",
    "Very disappointed with the quality.",
    "Stopped working after a week.",
    "Not as described. Waste of money.",
    "Poor build quality and slow shipping.",
    "Would not recommend to anyone.",
    "I regret buying this.",
    "Broke on the first use.",
    "Customer service was unhelpful.",
    "It didn't work at all."
]

In [3]:
data = []
for _ in range(50):
    data.append({"reviews": random.choice(positive_reviews), "label": "POSITIVE"})
for _ in range(50):
    data.append({"reviews": random.choice(negative_reviews), "label": "NEGATIVE"})

In [4]:
random.shuffle(data)

In [5]:
df = pd.DataFrame(data)

print("--- Verisetinin İlk 5 Satırı ---")
print(df.head())
print("\n" + "="*30 + "\n")

--- Verisetinin İlk 5 Satırı ---
                                      reviews     label
0       Poor build quality and slow shipping.  NEGATIVE
1           Not as described. Waste of money.  NEGATIVE
2                       I regret buying this.  NEGATIVE
3                      It didn't work at all.  NEGATIVE
4  Superb customer service and fast shipping.  POSITIVE




In [6]:
# Metin etiketlerini (POSITIVE/NEGATIVE) sayısal değerlere dönüştürme (1/0)
# Makine öğrenmesi modelleri sayısal verilerle çalışır.
df['label_num'] = df['label'].map({'POSITIVE': 1, 'NEGATIVE': 0})

In [7]:
# Bağımsız (X) ve bağımlı (y) değişkenleri ayırma
X = df['reviews']  # Özellikler (yorum metinleri)
y = df['label_num']  # Hedef (duygu etiketleri)

In [8]:
# Veriyi eğitim ve test setlerine ayırma
# Verinin %80'i modeli eğitmek, %20'si ise test etmek için kullanılacak.
# stratify=y, eğitim ve test setlerindeki pozitif/negatif oranının korunmasını sağlar.
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42, # Sonuçların tekrarlanabilir olması için
    stratify=y
)

In [9]:
# Kullanılacak vektörleştiricileri tanımlama
# Vektörleştiriciler, metinleri sayısal vektörlere dönüştürür.
vectorizers = {
    "CountVectorizer": CountVectorizer(), # Kelime sayımına göre vektörleştirme
    "TfidfVectorizer": TfidfVectorizer()  # Kelime frekansı-ters belge frekansına göre vektörleştirme
}

# Kullanılacak sınıflandırma modellerini tanımlama
models = {
    "Logistic Regression": LogisticRegression(random_state=42),
    "Multinomial Naive Bayes": MultinomialNB(),
    "Support Vector Classifier": SVC(random_state=42)
}

In [10]:
# Sonuçları saklamak için boş bir liste
results = []
# Eğitilmiş pipeline'ları saklamak için bir sözlük
pipelines = {}

print("--- Model Eğitimi ve Değerlendirmesi Başladı ---\n")

--- Model Eğitimi ve Değerlendirmesi Başladı ---



In [11]:
# Her bir vektörleştirici ve model kombinasyonunu denemek için döngü
for vec_name, vectorizer in vectorizers.items():
    for model_name, model in models.items():
        # Pipeline: Veri işleme adımlarını (vektörleştirme) ve modeli birleştirir.
        pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', model)
        ])

        # Modeli (pipeline'ı) eğitim verisiyle eğitme
        pipeline.fit(X_train, y_train)

        # Eğitilmiş pipeline'ı saklama
        pipeline_name = f"{vec_name}_{model_name}"
        pipelines[pipeline_name] = pipeline

        # Test seti üzerinde tahmin yapma
        y_pred = pipeline.predict(X_test)

        # Modelin doğruluk skorunu hesaplama
        accuracy = accuracy_score(y_test, y_pred)

        # Sonuçları listeye ekleme
        results.append({
            "Vektörleştirici": vec_name,
            "Model": model_name,
            "Doğruluk (%)": accuracy * 100
        })
        print(f"'{vec_name}' + '{model_name}' modeli eğitildi ve test edildi.")

print("\n--- Model Performans Karşılaştırması ---\n")

'CountVectorizer' + 'Logistic Regression' modeli eğitildi ve test edildi.
'CountVectorizer' + 'Multinomial Naive Bayes' modeli eğitildi ve test edildi.
'CountVectorizer' + 'Support Vector Classifier' modeli eğitildi ve test edildi.
'TfidfVectorizer' + 'Logistic Regression' modeli eğitildi ve test edildi.
'TfidfVectorizer' + 'Multinomial Naive Bayes' modeli eğitildi ve test edildi.
'TfidfVectorizer' + 'Support Vector Classifier' modeli eğitildi ve test edildi.

--- Model Performans Karşılaştırması ---



In [12]:
# Sonuçları DataFrame olarak düzenleme ve doğruluk oranına göre sıralama
results_df = pd.DataFrame(results).sort_values(by="Doğruluk (%)", ascending=False).reset_index(drop=True)
print(results_df.to_string())
print("\n" + "="*30 + "\n")


örnek_cümle = "The product is not good, it was a total waste of money."

print(f"Örnek Cümle: '{örnek_cümle}'\n")

# Tüm modellerin bu cümle için tahminlerini görelim
print("--- Tüm Modellerin Örnek Cümle İçin Tahminleri ---\n")
for pipeline_name, pipeline in pipelines.items():
    prediction = pipeline.predict([örnek_cümle])
    predicted_label = "POSITIVE" if prediction[0] == 1 else "NEGATIVE"
    print(f"{pipeline_name.replace('_', ' + '):<45}: {predicted_label}")

   Vektörleştirici                      Model  Doğruluk (%)
0  CountVectorizer        Logistic Regression         100.0
1  CountVectorizer    Multinomial Naive Bayes         100.0
2  CountVectorizer  Support Vector Classifier         100.0
3  TfidfVectorizer        Logistic Regression         100.0
4  TfidfVectorizer    Multinomial Naive Bayes         100.0
5  TfidfVectorizer  Support Vector Classifier         100.0


Örnek Cümle: 'The product is not good, it was a total waste of money.'

--- Tüm Modellerin Örnek Cümle İçin Tahminleri ---

CountVectorizer + Logistic Regression        : NEGATIVE
CountVectorizer + Multinomial Naive Bayes    : NEGATIVE
CountVectorizer + Support Vector Classifier  : NEGATIVE
TfidfVectorizer + Logistic Regression        : NEGATIVE
TfidfVectorizer + Multinomial Naive Bayes    : NEGATIVE
TfidfVectorizer + Support Vector Classifier  : NEGATIVE


In [13]:
# Sonuçları DataFrame olarak düzenleme ve doğruluk oranına göre sıralama
results_df = pd.DataFrame(results).sort_values(by="Doğruluk (%)", ascending=False).reset_index(drop=True)
print(results_df.to_string())
print("\n" + "="*30 + "\n")


örnek_cümle = "The product is good, but so expensive"

print(f"Örnek Cümle: '{örnek_cümle}'\n")

# Tüm modellerin bu cümle için tahminlerini görelim
print("--- Tüm Modellerin Örnek Cümle İçin Tahminleri ---\n")
for pipeline_name, pipeline in pipelines.items():
    prediction = pipeline.predict([örnek_cümle])
    predicted_label = "POSITIVE" if prediction[0] == 1 else "NEGATIVE"
    print(f"{pipeline_name.replace('_', ' + '):<45}: {predicted_label}")

   Vektörleştirici                      Model  Doğruluk (%)
0  CountVectorizer        Logistic Regression         100.0
1  CountVectorizer    Multinomial Naive Bayes         100.0
2  CountVectorizer  Support Vector Classifier         100.0
3  TfidfVectorizer        Logistic Regression         100.0
4  TfidfVectorizer    Multinomial Naive Bayes         100.0
5  TfidfVectorizer  Support Vector Classifier         100.0


Örnek Cümle: 'The product is good, but so expensive'

--- Tüm Modellerin Örnek Cümle İçin Tahminleri ---

CountVectorizer + Logistic Regression        : NEGATIVE
CountVectorizer + Multinomial Naive Bayes    : POSITIVE
CountVectorizer + Support Vector Classifier  : NEGATIVE
TfidfVectorizer + Logistic Regression        : NEGATIVE
TfidfVectorizer + Multinomial Naive Bayes    : POSITIVE
TfidfVectorizer + Support Vector Classifier  : POSITIVE


CountVectorizer + Multinomial Naive Bayes , TfidfVectorizer + Multinomial Naive Bayes , TfidfVectorizer + Support Vector Classifier  basit ama karışık bir cümlede yanıldı.